In [1]:
import arcpy
import pandas as pd
from collections import namedtuple

In [2]:
counties_dbf = r"C:\gis\Projects\HistoricCounties\Data\HistoricalCountyBoundaries\UT_Historical_Counties\UT_Historical_Counties.dbf"
districts_csv = r"C:\gis\Projects\HistoricCounties\Data\JudicialDistricts\test.csv"
counties_shp = r"C:\gis\Projects\HistoricCounties\Data\HistoricalCountyBoundaries\UT_Historical_Counties\UT_Historical_Counties.shp"


In [20]:
counties_list = []
county_fields = [f.name for f in arcpy.ListFields(counties_shp)]
with arcpy.da.SearchCursor(counties_shp, county_fields) as sc:
    for row in sc:
        counties_list.append(dict(zip(county_fields, row)))
    

In [21]:
counties_df = pd.DataFrame(counties_list)

In [23]:
counties_df.columns

Index(['FID', 'Shape', 'NAME', 'ID', 'STATE', 'FIPS', 'VERSION', 'START_DATE',
       'END_DATE', 'CHANGE', 'CITATION', 'START_N', 'END_N', 'AREA_SQMI',
       'DATASET', 'CNTY_TYPE', 'FULL_NAME'],
      dtype='object')

In [94]:
counties_df['START_DATE']

0     1849-07-02
1     1850-10-05
2     1850-12-03
3     1850-10-05
4     1850-10-05
         ...    
208   1921-03-10
209   1931-03-23
210   1931-03-23
211   2003-01-01
212   2003-01-01
Name: START_DATE, Length: 213, dtype: datetime64[ns]

In [3]:
districts_df = pd.read_csv(r"C:\gis\Projects\HistoricCounties\Data\JudicialDistricts\test.csv")
districts_df

,CountyName,StartDate,EndDate,NewDistrict,OldDistrict,Version
0,Beaver,1856-01-17,1859-01-20,2,2,1
1,Beaver,1859-01-21,1862-01-16,1,2,2
2,Beaver,1862-01-17,1896-01-05,2,1,3
3,Beaver,1896-01-06,2030-01-01,5,2,4
4,Box Elder,1856-01-17,1859-01-20,1,1,1
...,...,...,...,...,...,...
113,Richland,1865-12-27,NaN,3,3,1
114,Rio Virgen,1869-02-18,NaN,2,2,1
115,St. Marys,1856-01-17,1859-01-20,3,3,1
116,St. Marys,1859-01-21,NaN,2,3,2


In [4]:
districts_df['StartDate'] = pd.to_datetime(districts_df['StartDate'])
districts_df['EndDate'] = pd.to_datetime(districts_df['EndDate'])
districts_df['district_key']=districts_df['CountyName'].str.casefold() + '_d' + districts_df['Version'].astype(str)  #: add district key
districts_df['StartDate']


0     1856-01-17
1     1859-01-21
2     1862-01-17
3     1896-01-06
4     1856-01-17
         ...    
113   1865-12-27
114   1869-02-18
115   1856-01-17
116   1859-01-21
117   1856-01-17
Name: StartDate, Length: 118, dtype: datetime64[ns]

In [13]:
districts_df.dtypes

CountyName              object
StartDate       datetime64[ns]
EndDate         datetime64[ns]
NewDistrict             object
OldDistrict             object
Version                float64
district_key            object
dtype: object

In [32]:
counties_df['county_key']=counties_df['ID'] + '_S' + counties_df['VERSION'].astype(str)

In [15]:
counties_df[counties_df['ID'].str.contains('beaver')]

,FID,Shape,NAME,ID,STATE,FIPS,VERSION,START_DATE,END_DATE,CHANGE,CITATION,START_N,END_N,AREA_SQMI,DATASET,CNTY_TYPE,FULL_NAME,county_key
56,56,"(-112.17937602433766, 38.32630597762866)",BEAVER,uts_beaver,UT,49001,1,1856-01-05,1861-01-17,BEAVER created by Utah Territory from IRON; BE...,"(Utah Terr. Laws 1855, 5th sess., sec. 9/p. 7)",18560105,18610117,15141.0,UT_Historical_Counties,County,BEAVER,uts_beaver_S1
68,68,"(-112.03271613528274, 38.325452153781306)",BEAVER,uts_beaver,UT,49001,3,1861-02-28,1862-01-16,The United States created Colorado Territory f...,"(U.S. Stat., vol. 12, ch. 59[1861]/pp. 172-177...",18610228,18620116,7803.0,UT_Historical_Counties,County,BEAVER,uts_beaver_S3
78,78,"(-110.7025292913094, 38.32662877359086)",BEAVER,uts_beaver,UT,49001,2,1861-01-18,1861-02-27,"BEAVER lost to HUMBOLDT (Utah Territory, extin...","(Utah Terr. Laws 1861, 10th sess., p. 19)",18610118,18610227,11279.0,UT_Historical_Counties,County,BEAVER,uts_beaver_S2
86,86,"(-112.0228053482935, 38.363273470514734)",BEAVER,uts_beaver,UT,49001,5,1862-07-14,1865-01-15,BEAVER lost to Nevada Territory in present Nev...,"(U.S. Stat., vol. 12, ch. 173 [1862], p. 575; ...",18620714,18650115,9483.0,UT_Historical_Counties,County,BEAVER,uts_beaver_S5
100,100,"(-112.5151993561241, 38.36318198233733)",BEAVER,uts_beaver,UT,49001,4,1862-01-17,1862-07-13,"BEAVER gained from MILLARD, SANPETE, and ST. M...","(Utah Terr. Laws 1861, 11th sess., sec. 3/p. 47)",18620117,18620713,11049.0,UT_Historical_Counties,County,BEAVER,uts_beaver_S4
120,120,"(-113.70827225556754, 38.3589546264961)",BEAVER,uts_beaver,UT,49001,6,1865-01-16,1866-05-04,BEAVER lost to creation of PIUTE.,"(Utah Terr. Laws 1864, 14th sess., p. 16)",18650116,18660504,4094.0,UT_Historical_Counties,County,BEAVER,uts_beaver_S6
124,124,"(-113.23575592353453, 38.35770859461383)",BEAVER,uts_beaver,UT,49001,7,1866-05-05,2003-12-31,BEAVER lost to the state of Nevada when Nevada...,"(U.S. Stat., vol. 14, ch. 73[1866]/p. 43; Van ...",18660505,20031231,2592.0,UT_Historical_Counties,County,BEAVER,uts_beaver_S7


In [16]:
class County:
    def __init__(self, name):
        self.name = name.casefold()
        self.shape_df = None  #: All entries for this county in the historical county shape/geography dataframe
        self.district_df = None  #: All entries for this county in the historical dsitrict composition dataframe
        self.all_dates = None  #: Series of start dates day from both shape_df and district_df
        
    def setup(self, counties_df, districts_df):
        #: create a local shape dataframe of just the county, change the index to the start date of each version
        self.shape_df = counties_df[counties_df['ID'].str.contains(self.name)].copy()  #: copy to avoid chained indexing
        self.shape_df.set_index('START_DATE', inplace=True)
        self.shape_df.sort_index(inplace=True)

        #: create a local district dataframe of just the county, change the index to the start date of each version
        self.district_df = districts_df[districts_df['CountyName'].str.contains(self.name, case=False)].copy()
        self.district_df.set_index('StartDate', inplace=True)
        self.district_df.sort_index(inplace=True)

        #: create series of dates to check 
        self.all_dates = pd.Series(self.shape_df.index.union(self.district_df.index))
        
        
    def get_versions(self):
#         check_dates = self.all_dates + pd.Timedelta(days=1)
#         versions_df = pd.DataFrame(self.all_dates + pd.Timedelta(days=1))
        pass
        

In [17]:
class ChangeDate:
    def __init__(self, date):
        self.date = date
        self.county_version = 'n/a'
        self.district = 'n/a'
        
    def __repr__(self):
        out_string = ', '.join([str(self.date), self.county_version, str(self.district)])
        return out_string

In [18]:
beaver = County('Beaver')
beaver.setup(counties_df, districts_df)

dates = list(beaver.shape_df.index.union(beaver.district_df.index))
change_dates = [ChangeDate(date) for date in dates]
for change_date in change_dates:
    for county_row in beaver.shape_df.itertuples():
        if change_date.date >= county_row.Index and change_date.date <= county_row.END_DATE:
            change_date.county_version = county_row.county_key
            break
    for district_row in beaver.district_df.itertuples():
        if change_date.date >= district_row.Index and change_date.date <= district_row.EndDate:
            change_date.district = district_row.NewDistrict
            break
change_dates

#: Have to convert to namedtuple afterwards so that we can alter the list of ChangeDate items as we iterate
ChangeDatesTuple = namedtuple('ChangeDatesTuple', 'date county_version district')
new_dates = [ChangeDatesTuple(cd.date, cd.county_version, cd.district) for cd in change_dates]
new_dates
dates_df = pd.DataFrame(new_dates)
dates_df
    

,date,county_version,district
0,1856-01-05,uts_beaver_S1,n/a
1,1856-01-17,uts_beaver_S1,2
2,1859-01-21,uts_beaver_S1,1
3,1861-01-18,uts_beaver_S2,1
4,1861-02-28,uts_beaver_S3,1
5,1862-01-17,uts_beaver_S4,2
6,1862-07-14,uts_beaver_S5,2
7,1865-01-16,uts_beaver_S6,2
8,1866-05-05,uts_beaver_S7,2
9,1896-01-06,uts_beaver_S7,5


In [54]:
all_times + pd.Timedelta(days=1)

0   1856-01-06
1   1856-01-18
2   1859-01-22
3   1861-01-19
4   1861-03-01
5   1862-01-18
6   1862-07-15
7   1865-01-17
8   1866-05-06
9   1896-01-07
dtype: datetime64[ns]

In [19]:
dfs_by_county = {}
for county in counties_df['ID'].unique():
    county_name = county.split('_')[-1]
    dfs_by_county[county_name] = counties_df[counties_df['ID'] == county]

In [20]:
dfs_by_county

{'des':     FID                                     Shape     NAME   ID STATE FIPS  \
 0     0  (-113.45238337501421, 38.03305439561905)  Deseret  des    UT        
 5     5  (-113.48001976528437, 38.00695181641287)  Deseret  des              
 13   13  (-113.47478944961205, 38.00598435115826)  Deseret  des              
 
     VERSION START_DATE   END_DATE  \
 0         1 1849-07-02 1850-01-30   
 5         3 1850-12-03 1851-04-04   
 13        2 1850-01-31 1850-12-02   
 
                                                CHANGE  \
 0   The General Assembly of the newly proposed sta...   
 5   The state of Deseret lost non-county area to I...   
 13  The state of Deseret lost non-county area to c...   
 
                                              CITATION   START_N     END_N  \
 0   (Atlas of Utah, 160-161; Brown, Cannon, and Ja...  18490702  18500130   
 5                                (Morgan, 56-57, 157)  18501203  18510404   
 13                              (Morgan, pp. 180-181

In [25]:
for c in counties_df['ID'].unique():
    print(c)

des
uts_saltlake
uts_iron
uts_davis
uts_weber
uts_utah
uts_sanpete
uts_tooele
utt
uts_millard
uts_washington
uts_juab
utt_desert
utt_greenriver
uts_summit
utt_carson
utt_shambip
utt_cedar
uts_cache
uts_boxelder
utt_malad
uts_beaver
utt_humboldt
utt_greasewood
utt_stmarys
jet_mountain
jet_saratoga
jet_part
uts_morgan
uts_wasatch
uts_kane
uts_rich
uts_uintah
uts_sevier
uts_piute
utt_riovirgen
uts_sanjuan
uts_emery
uts_garfield
uts_grand
uts_wayne
uts_carbon
uts_duchesne
uts_daggett


In [37]:
districts_df['CountyName'].unique()

array(['Beaver', 'Box Elder', 'Cache', 'Carbon', 'Daggett', 'Davis',
       'Duchesne', 'Emery', 'Garfield', 'Grand', 'Iron', 'Juab', 'Kane',
       'Millard', 'Morgan', 'Piute', 'Rich', 'Salt Lake', 'San Juan',
       'Sanpete', 'Sevier', 'Summit', 'Tooele', 'Uintah', 'Utah',
       'Wasatch', 'Washington', 'Wayne', 'Weber', 'Carson', 'Cedar',
       'Desert', 'Greasewood', 'Green River', 'Humboldt', 'Malad',
       'Richland', 'Rio Virgen', 'St. Marys', 'Shambip'], dtype=object)

In [28]:
def clean_county_name(name):
    clean_name = name.casefold()
    clean_name = clean_name.replace(' ', '')
    clean_name = clean_name.replace('.', '')
#     if clean_name == 'richland':  #: custom to handle Richland renamed to Rich
#         clean_name = 'rich'
        
    return clean_name

In [33]:
for county in districts_df['CountyName'].unique():
    county_key = clean_county_name(county)
    if county_key in dfs_by_county:
        print(dfs_by_county[county_key]['county_key'])

56     uts_beaver_S1
68     uts_beaver_S3
78     uts_beaver_S2
86     uts_beaver_S5
100    uts_beaver_S4
120    uts_beaver_S6
124    uts_beaver_S7
Name: county_key, dtype: object
52     uts_boxelder_S1
88     uts_boxelder_S3
105    uts_boxelder_S2
126    uts_boxelder_S4
158    uts_boxelder_S5
Name: county_key, dtype: object
49     uts_cache_S1
99     uts_cache_S2
107    uts_cache_S3
Name: county_key, dtype: object
186    uts_carbon_S1
Name: county_key, dtype: object
199    uts_daggett_S1
200    uts_daggett_S2
Name: county_key, dtype: object
3      uts_davis_S1
27     uts_davis_S2
39     uts_davis_S3
44     uts_davis_S4
93     uts_davis_S5
159    uts_davis_S6
Name: county_key, dtype: object
191    uts_duchesne_S1
194    uts_duchesne_S2
Name: county_key, dtype: object
153    uts_emery_S2
162    uts_emery_S1
176    uts_emery_S3
187    uts_emery_S4
211    uts_emery_S5
Name: county_key, dtype: object
164    uts_garfield_S1
168    uts_garfield_S2
Name: county_key, dtype: object
177    uts_gr

In [ ]:
def in_date_range(start_date, end_date):
    if 

In [ ]:
for county_name in districts_df['CountyName'].unique():
    county_dict_key = county_name.casefold().replace(' ', '') 
    if county_dict_key in dfs_by_county:
        for df in dfs_by_county[county_dict_key]:

In [40]:
'des'.split('_')

['des']

In [42]:
'ut_saltlake'.split('_')

['ut', 'saltlake']

In [50]:
'salt lake'.replace(' ', '')

'saltlake'

In [2]:
test_df = pd.DataFrame(data={'county_key':['uts_co_S1', 'uts_co_s2', 'uts_co_s3'], 'END_DATE':['2020-02-19', '2020-02-24', '2020-02-28']}, index=['2020-02-05', '2020-02-20', '2020-02-25'])
test_df.index = pd.to_datetime(test_df.index)
test_df['END_DATE'] = pd.to_datetime(test_df['END_DATE'])
test_df.index

DatetimeIndex(['2020-02-05', '2020-02-20', '2020-02-25'], dtype='datetime64[ns]', freq=None)

In [3]:
test_df

,county_key,END_DATE
2020-02-05,uts_co_S1,2020-02-19
2020-02-20,uts_co_s2,2020-02-24
2020-02-25,uts_co_s3,2020-02-28


In [4]:
shape_df = pd.DataFrame(
    data={
        'county_key': ['uts_co_S1', 'uts_co_s2', 'uts_co_s3'],
        'END_DATE': ['2020-02-19', '2020-02-24', '2020-02-28'],
    },
    index=['2020-02-05', '2020-02-20', '2020-02-25']
)
shape_df.index = pd.to_datetime(shape_df.index)
shape_df['END_DATE'] = pd.to_datetime(shape_df['END_DATE'])

districts_df = pd.DataFrame(
    data={
        'NewDistrict': ['1', '2', '3'],
        'EndDate': ['2020-02-14', '2020-02-24', '2020-02-28'],
    },
    index=['2020-02-10', '2020-02-15', '2020-02-25']
)
districts_df.index = pd.to_datetime(districts_df.index)
districts_df['EndDate'] = pd.to_datetime(districts_df['EndDate'])

In [6]:
districts_df

,NewDistrict,EndDate
2020-02-10,1,2020-02-14
2020-02-15,2,2020-02-24
2020-02-25,3,2020-02-28


In [8]:
pd.Series(shape_df.index.union(districts_df.index))

0   2020-02-05
1   2020-02-10
2   2020-02-15
3   2020-02-20
4   2020-02-25
dtype: datetime64[ns]

In [14]:
districts_df['OldDistrict'] = ['10', '1', '2']

In [15]:
districts_df

,NewDistrict,EndDate,OldDistrict
2020-02-10,1,2020-02-14,10
2020-02-15,2,2020-02-24,1
2020-02-25,3,2020-02-28,2


In [5]:
districts_df.OldDistrict.eq(districts_df.NewDistrict.shift())

0      False
1       True
2       True
3       True
4      False
       ...  
113     True
114    False
115    False
116     True
117    False
Length: 118, dtype: bool

In [6]:
districts_df.NewDistrict.shift()

0      NaN
1        2
2        1
3        2
4        5
      ... 
113      3
114      3
115      2
116      3
117      2
Name: NewDistrict, Length: 118, dtype: object

In [12]:
# districts_df.set_index('StartDate', inplace=True)
districts_df.sort_index(inplace=True)

districts_df.index.eq(districts_df['EndDate'].shift() + pd.Timedelta(days=1))

TypeError: 'DatetimeIndex' object is not callable

In [18]:
districts_df.loc[:, ['CountyName', 'EndDate']]

,CountyName,EndDate
StartDate,,
1851-01-08,Salt Lake,1856-01-16
1851-01-08,Tooele,1856-01-16
1851-01-08,Utah,1851-10-03
1851-08-08,Iron,1855-01-18
1851-08-08,Davis,1851-10-03
...,...,...
1982-02-16,Millard,2030-01-01
1982-02-16,Garfield,2030-01-01
1988-04-25,Duchesne,2030-01-01
